## Examples

In [9]:
!python -m pip install numpy

In [10]:
!python -m pip uninstall -y changepoint_online

Found existing installation: changepoint-online 0.0.1
Uninstalling changepoint-online-0.0.1:
  Successfully uninstalled changepoint-online-0.0.1


In [11]:
!python -m pip install '../package/'

Processing /home/romano/Dropbox/changepoint.online/python/package
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for changepoint-online: filename=changepoint_online-0.0.1-py3-none-any.whl size=17195 sha256=06fb5b500f9bb62e7d0ca44e43d93bc00328546dd720d6fbbfcc3f81e255a264
  Stored in directory: /tmp/pip-ephem-wheel-cache-eyx7trlq/wheels/ef/98/3a/e389858773e4fe5f47ec0415eb44c05e2319de69019c54a854
Successfully built changepoint-online


In [12]:
from changepoint_online import *

In [13]:
help(Focus)

Help on class Focus in module changepoint_online.focus:

class Focus(builtins.object)
 |  Focus(comp_func)
 |  
 |  The Focus class implements the FOCuS method, an algorithm for detecting changes in data streams on one-parameter exponential family models.
 |  For instance, FOCuS can detect changes-in-mean in a Gaussian data stream (white noise). 
 |  It can be applied to settings where either the pre-change parameter is known or unknown.
 |      
 |  FOCuS solves the CUSUM likelihood-ratio test exactly in O(log(n)) time per iteration, where n is the current iteration. 
 |  The method is equivalent to running a rolling window (MOSUM) simultaneously for all sizes of window, or the Page-CUSUM for all possible values
 |  of the size of change (an infinitely dense grid). 
 |  
 |  DISCLAIMER: Albeit the FOCuS algorithm decreases the per-iteration cost from O(n) to O(log(n)), this 
 |  implementation is not technically online as for n->infty this code would inherently overflow. 
 |  True onl

In [14]:
import numpy as np

In [15]:
np.random.seed(0)
Y = np.concatenate((np.random.normal(loc=0.0, scale=1.0, size=5000), np.random.normal(loc=10.0, scale=1.0, size=5000)))

### Guassian

In [16]:
detector = Focus(Gaussian(theta0=0))

In [17]:
threshold = 10.0
for y in Y:
    detector.update(y)
    if detector.statistic() >= threshold:
        break

In [18]:
detector.changepoint()

{'stopping_time': 5001, 'changepoint': 5000}

### Poisson

In [26]:
np.random.seed(0)
Y = np.concatenate((np.random.poisson(lam=4.0, size=5000), np.random.poisson(lam=4.2, size=5000)))

In [27]:
detector = Focus(Poisson())

In [29]:
threshold = 10.0
for y in Y:
    detector.update(y)
    if detector.statistic() >= threshold:
        break

In [ ]:
detector.changepoint()

### Gamma

In [30]:
np.random.seed(0)
Y = np.concatenate((np.random.gamma(4.0, scale=3.0, size=5000), np.random.gamma(4.0, scale=6.0, size=5000)))

In [31]:
detector = Focus(Gamma(shape=4.0))

In [32]:
threshold = 10.0
for y in Y:
    detector.update(y)
    if detector.statistic() >= threshold:
        break

In [33]:
detector.changepoint()

{'stopping_time': 1, 'changepoint': 0}